In [1]:
import requests, os, re
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
# Create list of URLs of the APIs to call in order to get pas season stats
years = list(range(1990,2024))

In [ ]:
# Function to get JSON data
def get_data(url):
    response = requests.get(url)
    # Check the response status code
    if response.status_code == 200:
        # If successful, try parsing as JSON
        try:
            data = response.json()
        except requests.exceptions.JSONDecodeError:
            print("Response content is not valid JSON:")
            print(response.text)
    else:
        # If error, print status code and content
        print(f"Request failed with status code: {response.status_code}")
        data = response.text

    return data

In [ ]:
# Function to make a pandas dataframe with the stats
def create_df(data, year):
    # Get the table headers and stats from the JSON data
    headers = data['resultSet']['headers']
    stats = data['resultSet']['rowSet']
    # Create the dataframe
    df = pd.DataFrame(stats, columns=headers)
    df['Year'] = year
    return df

In [ ]:
# Create a folder to store all the data
os.makedirs("nba_stats", exist_ok=True)

# Loop through api_urls and create CSV files
for year in years:
  data = get_data(f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season={year}-{str(year+1)[-2:]}&SeasonType=Regular%20Season&StatCategory=PTS")
  df = create_df(data, year)
  df.to_csv(f"nba_stats/{year}-{str(year+1)[-2:]}.csv", index=False)